# Equity Examples
(c) Marek Ozana

Example usage of polars-bloomberg in Equity space

In [1]:
# imports
import altair as alt

from polars_bloomberg import BQuery


## S&P Annual Price Returns

In [2]:
# get S&P annual price returns
bin_names: list = [
    "< -50",
    "-50 to -40",
    "-40 to -30",
    "-30 to -20",
    "-20 to -10",
    "-10 to 0",
    "0 to 10",
    "10 to 20",
    "20 to 30",
    "30 to 40",
    "40 to 50",
    "> 50",
]
bins = list(range(-50, 60, 10))

query = f"""
let(
    #rets = dropna(pct_diff(px_last(dates=range(-100Y, 0Y), per=Y)));
    #bins = bins(#rets, {bins}, bin_names={bin_names});
)
get(#rets, #bins)
for(['SPX Index'])
"""
with BQuery() as bq:
    df = bq.bql(query).combine()

# Create Chart
base = alt.Chart(df, title="S&P 500 Annual Price Returns").encode(
    x=alt.X("#bins:O", title="Annual Returns (%)").sort(bin_names),
    y=alt.Y("count()", title="Count").stack("zero"),
)
bars = base.mark_bar().encode(
    color=alt.Color("year(DATE):O").scale(scheme="viridis").legend(None)
)
txt = base.mark_text(dy=9).encode(
    text=alt.Text("year(DATE):O"),
)

(bars + txt).properties(width=600, height=400).configure_axisX(labelAngle=0)

alt.LayerChart(...)